In [ ]:
# 1. Star
# 2. Folk
# 3. Watching
# 4. Commits
# 5. Issue
# 6. PL (Programming Language used by the OSS)
# 7. Pull Request
# 8. Actions (I assume that unit tests is a part of Action. Or I may)
# 9. Docs (Readme.md, Governance.md, Open source license.md, Security.md)
# 10. Contributor
# 11. Code

In [15]:
import datetime
import pandas as pd
import os
import json

In [ ]:
# the metrics calculated will be saved in '../metrics.csv'

In [18]:
repos_data_path = '/Users/zhangyujin/Desktop/repos-data'
metrics_path = '../metrics.csv'
metrics = pd.read_csv(metrics_path)

In [ ]:
# 01 repo_size

In [23]:
metrics['stars'] = None
metrics['forks'] = None
for index, row in metrics.iterrows():
    owner = row['owner']
    repo = row['repo']
    repo_path = f'{repos_data_path}/{owner}_{repo}'
    if os.path.exists(repo_path):
        repo_info_path = f'{repos_data_path}/{owner}_{repo}/repo.json'
        if os.path.exists(repo_info_path):
            with open(repo_info_path, 'r') as f:
                repo_info = json.load(f)
            stars = repo_info.get('stargazers_count', 0)
            forks = repo_info.get('forks_count', 0)
            metrics.at[index, 'stars'] = stars
            metrics.at[index, 'forks'] = forks

In [24]:
metrics.to_csv(metrics_path, index=False)